
# Perspectief 2 – Beperkte impact van EV‑adoptie
Dit hoofdstuk bekijkt in hoeverre de huidige adoptie van elektrische voertuigen (EV’s)
**niet altijd** resulteert in een uniforme of significante daling van de CO₂‑uitstoot
van nieuw geregistreerde auto’s.

We presenteren twee visualisaties:

1. **Lijngrafiek** – Evolutie van CO₂‑uitstoot per land (2013‑2023)  
2. **Interactieve kaart** – EV‑aandeel per land met jaarslider


In [1]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

cars = pd.read_csv('../data/cars_by_fuel.csv')
co2  = pd.read_csv('../data/co2_new_cars.csv')

# --- merge & EV‑share ---
tmp = {}
for _, r in cars.iterrows():
    key = f"{r.geo}-{r.TIME_PERIOD}"
    tmp.setdefault(key, {'geo': r.geo, 'year': int(r.TIME_PERIOD),
                         'elc': 0, 'total': 0})
    if r.mot_nrg == 'ELC':
        tmp[key]['elc'] = r.OBS_VALUE
    elif r.mot_nrg == 'TOTAL':
        tmp[key]['total'] = r.OBS_VALUE

for _, r in co2.iterrows():
    key = f"{r.geo}-{r.TIME_PERIOD}"
    if key in tmp:
        tmp[key]['co2'] = r.OBS_VALUE

df = (pd.DataFrame(tmp.values())
        .dropna(subset=['co2'])
        .assign(evShare=lambda d: d.elc / d.total * 100))

focus_codes = ['NL','DE','NO','PL','FR','ES']
focus = df[df.geo.isin(focus_codes)]


## Visualisatie 2.1 – Evolutie CO₂‑uitstoot per land

In [2]:

fig_line = px.line(focus, x='year', y='co2', color='geo',
                   title='Gem. CO₂‑uitstoot nieuwe auto’s (2013‑2023)',
                   labels=dict(year='Jaar', co2='CO₂ (g/km)', geo='Land'))
fig_line.update_traces(mode='lines+markers')
fig_line.show()


Opvallend is dat alleen Noorwegen een **scherpe** daling laat zien; andere landen tonen een gematigder of wisselend patroon.

## Visualisatie 2.2 – EV‑aandeel per land (choropleth met slider)

In [3]:

iso_map = {'AT':'AUT','BE':'BEL','BG':'BGR','HR':'HRV','CY':'CYP','CZ':'CZE','DK':'DNK','EE':'EST',
           'FI':'FIN','FR':'FRA','DE':'DEU','EL':'GRC','HU':'HUN','IE':'IRL','IT':'ITA','LV':'LVA',
           'LT':'LTU','LU':'LUX','MT':'MLT','NL':'NLD','PL':'POL','PT':'PRT','RO':'ROU','SK':'SVK',
           'SI':'SVN','ES':'ESP','SE':'SWE','NO':'NOR','IS':'ISL','CH':'CHE','UK':'GBR'}

df['iso3'] = df.geo.map(iso_map)
rows = df.dropna(subset=['iso3'])

years = sorted(rows.year.unique())
zmax = rows.evShare.quantile(0.98)

frames = []
for yr in years:
    sub = rows[rows.year == yr]
    frames.append(go.Frame(name=str(yr),
                           data=[go.Choropleth(locations=sub.iso3,
                                               z=sub.evShare,
                                               text=sub.geo,
                                               colorscale='Viridis',
                                               zmin=0, zmax=zmax,
                                               hovertemplate='%{text}: %{z:.2f}%<extra></extra>')]))

fig_map = go.Figure(
    data=frames[0].data,
    frames=frames,
    layout=go.Layout(
        title=f'EV‑aandeel per land ({years[0]})',
        geo=dict(scope='europe', projection_type='natural earth',
                 showframe=False, showcountries=True),
        sliders=[dict(steps=[dict(method='animate', label=str(y),
                                  args=[[str(y)], dict(mode='immediate',
                                                       frame=dict(duration=500, redraw=False),
                                                       transition=dict(duration=300))])
                            for y in years],
                      currentvalue=dict(prefix='Jaar: '),
                      pad=dict(t=30))]
    ))
fig_map.show()


De kaart laat zien dat de **hoge EV‑penetratie** vrijwel exclusief is voor Noorwegen (en in mindere mate Nederland), terwijl een groot deel van Europa onder de 5 % blijft. Dat verklaart waarom de totale CO₂‑daling in vis. 2.1 beperkt is.